In [1]:
import os

In [2]:
%pwd

'/Users/ananyaghosh/Downloads/Kidney-Disease-Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/ananyaghosh/Downloads/Kidney-Disease-Classification'

In [ ]:
# import dagshub
# dagshub.init(repo_owner='anuz123', repo_name='Kidney_Disease_Project', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

# https://dagshub.com/anuz123/Kidney_Disease_Project.mlflow

In [ ]:
# %pip install dagshub

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 11.8 MB/s eta 0:00:00a 0:00:01
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 9.9 MB/s eta 0:00:00ta 0:00:01
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import dagshub
dagshub.init(repo_owner='anuz123', repo_name='Kidney_Disease_Project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

/Users/ananyaghosh/Downloads/Kidney-Disease-Classification/.venv/lib/python3.11/site-packages/rich/live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=7abb42c0-269b-4b54-8acc-2ba118ee44f1&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=f9f0667f68c95dae708d545b829365601232f95ce56fe628792c40cd5350625e




Accessing as anuz123

Initialized MLflow to track repo "anuz123/Kidney_Disease_Project"

Repository anuz123/Kidney_Disease_Project initialized!

🏃 View run sassy-ray-296 at: https://dagshub.com/anuz123/Kidney_Disease_Project.mlflow/#/experiments/0/runs/ab3276857801475fb0acb7793912e007
🧪 View experiment at: https://dagshub.com/anuz123/Kidney_Disease_Project.mlflow/#/experiments/0


In [ ]:
# os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/entbappy/Kidney-Disease-Classification-MLflow-DVC.mlflow"
# os.environ["MLFLOW_TRACKING_USERNAME"]="entbappy"
# os.environ["MLFLOW_TRACKING_PASSWORD"]="6824692c47a369aa6f9eac5b10041d5c8edbcef0"

In [8]:
import tensorflow as tf

In [9]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [16]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/anuz123/Kidney_Disease_Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [17]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [18]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-02-16 19:25:52,595: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-16 19:25:52,598: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-16 19:25:52,598: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.


/Users/ananyaghosh/Downloads/Kidney-Disease-Classification/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 34s 4s/step - accuracy: 0.3158 - loss: 16.3638
[2025-02-16 19:26:26,845: INFO: common: json file saved at: scores.json]


2025/02/16 19:26:27 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/02/16 19:26:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'VGG16Model'.
2025/02/16 19:26:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


🏃 View run classy-mouse-635 at: https://dagshub.com/anuz123/Kidney_Disease_Project.mlflow/#/experiments/0/runs/147edb2d26704453b353126ae20d9c51
🧪 View experiment at: https://dagshub.com/anuz123/Kidney_Disease_Project.mlflow/#/experiments/0
